In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/student_resource 3/src')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/student_resource 3/dataset')

In [ ]:
DATASET_FOLDER = '/content/drive/MyDrive/student_resource 3/dataset'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

In [38]:
import shutil
shutil.rmtree('/content/drive/MyDrive/student_resource 3/dataset/images')

In [ ]:
import os
import pandas as pd
import urllib.request
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split

# 1. Load and Sample the Dataset
data = pd.read_csv('/content/drive/MyDrive/student_resource 3/dataset/train.csv')  # Replace with actual dataset
data_sampled = data.sample(frac=0.01, random_state=42)  # Taking 1% of the data

# 2. Download the Images
if not os.path.exists('/content/drive/MyDrive/student_resource 3/dataset/images'):
    os.makedirs('/content/drive/MyDrive/student_resource 3/dataset/images')

def download_image(image_url, save_path):
    try:
        urllib.request.urlretrieve(image_url, save_path)
    except Exception as e:
        print(f"Failed to download {image_url}: {str(e)}")

for index, row in data_sampled.iterrows():
    image_url = row['image_link']
    save_path = os.path.join('images', f"{row['group_id']}.jpg")
    download_image(image_url, save_path)

# 3. Preprocessing and Dataset Class for PyTorch
class ImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.dataframe.iloc[idx]['group_id']}.jpg")
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        entity_name = self.dataframe.iloc[idx]['entity_name']
        entity_value = self.dataframe.iloc[idx]['entity_value']

        # Converting entity name to a numerical value if needed (use one-hot encoding or simple mapping)
        entity_value = self._convert_to_numeric(entity_value)  # This needs to be implemented based on your target

        return image, entity_name, entity_value

    def _convert_to_numeric(self, value):
        # Conversion logic: For example, you can convert weights/volumes to grams/milliliters
        try:
            if 'gram' in value:
                return float(value.split()[0])  # Simple example, you might need more complex parsing
            elif 'kilogram' in value:
                return float(value.split()[0]) * 1000
            elif 'milliliter' in value:
                return float(value.split()[0]) *4000
            elif 'liter' in value:
                return float(value.split()[0]) * 10
            elif 'gallob' in value:
                return float(value.split()[0]) * 3.785
            elif 'watt' in value:
                return float(value.split()[0]) * 7000
            elif 'pound' in value:
                return float(value.split()[0]) * 453.592
            elif 'ounce' in value:
                return float(value.split()[0]) * 28.3495
            elif 'volt' in value:
                return float(value.split()[0]) * 8000
            elif 'fluid ounce' in value:
                return float(value.split()[0]) * 2900.5735
            elif 'ton' in value:
                return float(value.split()[0]) * 100000
            elif 'inch' in value:
                return float(value.split()[0]) * 254
            elif 'milligram' in value:
                return float(value.split()[0]) * 20000
            elif 'microgram' in value:
                return float(value.split()[0]) *30000
            # Add more conversions for other units
            else:
                return 0.0
        except:
            return 0.0

# 4. Define Transforms and Load Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = ImageDataset(data_sampled, img_dir='images', transform=transform)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 5. Define the Model (CNN + Fully Connected for entity_name)
class CNNModel(nn.Module):
    def __init__(self, num_classes=1):  # We're predicting a single numeric value (entity_value)
        super(CNNModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 512)  # Adjust final layer for feature extraction

        self.fc_entity_name = nn.Linear(512, 256)  # Optional: If you want to use entity name in prediction
        self.fc_final = nn.Linear(512, num_classes)

    def forward(self, image, entity_name):
        image_features = self.cnn(image)  # CNN image features

        # Optional: If entity_name is numeric, you can concatenate it with image_features
        # entity_features = self.fc_entity_name(entity_name)
        # combined_features = torch.cat((image_features, entity_features), dim=1)

        output = self.fc_final(image_features)
        return output

# 6. Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
criterion = nn.MSELoss()  # Since we're predicting a continuous value (entity_value)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    images = images.float()
    entity_values = entity_values.float()
    for images, entity_names, entity_values in train_loader:
        images, entity_values = images.to(device), entity_values.to(device).float()  # Convert target to float

        optimizer.zero_grad()
        outputs = model(images, entity_names)
        loss = criterion(outputs, entity_values.unsqueeze(1))  # Ensure correct shape for MSE
        loss.backward()
        optimizer.step()


        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# 8. Testing Loop
model.eval()
with torch.no_grad():
    total_loss = 0.0
    for images, entity_names, entity_values in test_loader:
        images, entity_values = images.to(device), entity_values.to(device).float()  # Convert target to float

        outputs = model(images, entity_names)
        loss = criterion(outputs, entity_values.unsqueeze(1))
        total_loss += loss.item()

    print(f"Test Loss: {total_loss/len(test_loader)}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 1562505.8695803555
Epoch [2/10], Loss: 1551852.5506036931
Epoch [3/10], Loss: 1557471.4860617898
Epoch [4/10], Loss: 1557566.7972227153
